In [18]:
import torch
import torch.nn as nn
import torchvision 
from torchvision import transforms
from torch.utils.data import DataLoader

from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [19]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [20]:
train_path = "./train/"
test_path = "./test/"
val_path = "./val/"

In [21]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [22]:
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)

In [23]:
batch_size = 256
num_classes = 2

In [24]:
train_loader = DataLoader(dataset=train_data,batch_size=batch_size)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size)

In [25]:
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [26]:
class VGGNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),          
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        ) 
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),          
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        ) 
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),          
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),          
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        )
        self.block5 = nn.Sequential(
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),          
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3),padding=(1,1),stride=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        )

        self.flat = nn.Flatten()

        self.dense = nn.Sequential(
            nn.Linear(in_features=(7*7*512),out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096,out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096,out_features=num_classes),
            nn.Softmax()
        )

    def forward(self,x):
        x = self.block1(x) 
        x = self.block2(x) 
        x = self.block3(x) 
        x = self.block4(x) 
        x = self.block5(x)
        x = self.flat(x)
        x = self.dense(x)
        return x 

In [27]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device=device):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        lr_schduler.step()

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
model = VGGNet()

model.to(device)

In [29]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01, momentum=0.9,weight_decay=0.0005)

loss = nn.CrossEntropyLoss()

In [30]:
train(model=model, optimizer=optimizer,loss_fn=loss,train_loader=train_loader,val_loader=val_loader,epochs=20,device=device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.06 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 8.47 GiB is allocated by PyTorch, and 80.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)